In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/jason/opt/anaconda3/envs/gaze-track/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class eye_model(nn.Module):
    def __init__(self):
        super(eye_model, self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=0),
            nn.BatchNorm2d(32, momentum=0.9),
            nn.LeakyReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.02),
            
            nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(64, momentum=0.9),
            nn.LeakyReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.02),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(128, momentum=0.9),
            nn.LeakyReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.02),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [3]:
class landmark_model(nn.Module):
    def __init__(self):
        super(landmark_model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 128),
            nn.BatchNorm1d(128, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Linear(128, 16),
            nn.BatchNorm1d(16, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Linear(16, 16),
            nn.BatchNorm1d(16, momentum=0.9),
            nn.ReLU(inplace=True),
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
class gazetrack_model(nn.Module):
    def __init__(self):
        super(gazetrack_model, self).__init__()        
        self.eye_model = eye_model()
#         self.eye_model = torch.quantization.fuse_modules(self.eye_model, 
#                                                          [['model.0', 'model.1'],
#                                                           ['model.5', 'model.6'],
#                                                           ['model.10', 'model.11']])
        self.lmModel = landmark_model()
        self.combined_model = nn.Sequential(nn.Linear(512+512+16, 8),
                                            nn.BatchNorm1d(8, momentum=0.9),
                                            nn.Dropout(0.12),
                                            nn.ReLU(inplace = True),
                                            nn.Linear(8, 4),
                                            nn.BatchNorm1d(4, momentum=0.9),
                                            nn.ReLU(inplace = True),
                                            nn.Linear(4, 2),)

    def forward(self, leftEye, rightEye, lms):
        l_eye_feat = torch.flatten(self.eye_model(leftEye), 1)
        r_eye_feat = torch.flatten(self.eye_model(rightEye), 1)
        
        lm_feat = self.lmModel(lms)
        
        combined_feat = torch.cat((l_eye_feat, r_eye_feat, lm_feat), 1)
        out = self.combined_model(combined_feat)
        return out

In [6]:
model = gazetrack_model()
if(torch.cuda.is_available()):
    dev = torch.device('cuda:0')
else:
    dev = torch.device('cpu')

print(dev)
    
weights = torch.load("checkpoints/GoogleCheckpoint_GoogleSplit.ckpt", map_location=dev)['state_dict']
model.load_state_dict(weights)
model.to(dev)
model.eval()
print(model)

cpu


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/GoogleCheckpoint_MITSplit.ckpt'

## Fuse Layers
https://pytorch.org/docs/stable/generated/torch.quantization.fuse_modules.html

conv, bn
conv, bn, relu
conv, relu
linear, relu
bn, relu

Don't know if fusing is 

In [18]:
modules_to_fuse = []
# modules_to_fuse.append(['eye_model.0', 'eye_model.1', 'eye_model.2'])
# modules_to_fuse.append(['eye_model.5', 'eye_model.6', 'eye_model.7'])
# modules_to_fuse.append(['eye_model.10', 'eye_model.11', 'eye_model.12'])
# modules_to_fuse.append(['eye_model.0', 'eye_model.1', 'eye_model.2'])

# modules_to_fuse.append(['lmModel.0', 'lmModel.1', 'lmModel.2'])
# modules_to_fuse.append(['lmModel.3', 'lmModel.4', 'lmModel.5'])
# modules_to_fuse.append(['lmModel.6', 'lmModel.7', 'lmModel.8'])

# modules_to_fuse.append(['combined_model.5', 'combined_model.6'])
modules_to_fuse.append(['conv_bn_relu.conv', 'conv_bn_relu.bn'])
torch.quantization.fuse_modules(model, modules_to_fuse, inplace=True)
print(model)

AttributeError: 'gazetrack_model' object has no attribute 'conv_bn_relu'

## Quantize

Static post-training quantization

In [19]:
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')

torch.quantization.prepare(model, inplace=True)

# use data set to calibrate quantized weights
def calibrate(model, calibration_data): 
    return

calibrate(model, [])

model = torch.quantization.convert(model, inplace=True)

/Users/jason/opt/anaconda3/envs/gaze-track/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1135: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


## Save Model

In [21]:
from torch.utils.mobile_optimizer import optimize_for_mobile

torchscript_model = torch.jit.script(model)

torchscript_model_optimized = optimize_for_mobile(torchscript_model)

torch.jit.save(torchscript_model_optimized, 'model.pt')